In [ ]:
!pip install spark-nlp
!pip install pyspark

In [ ]:

from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, regexp_replace, trim
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from pyspark.ml import PipelineModel
from sklearn.metrics import classification_report
import re
import json
import requests
import pandas as pd


In [ ]:
!pip install flask_ngrok
!pip install pyngrok==4.1.1
!ngrok authtoken 2SxmTug3FdhBXFEtCQYJxtdKAao_31ziasGpNRxkJr47xdhVn

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
spark = sparknlp.start()
spark = SparkSession.builder.appName("sentimentanalysis").getOrCreate()

In [ ]:
spark.stop()

In [ ]:
from flask import render_template, Flask, request
import pandas as pd
from flask_ngrok import run_with_ngrok
templates_folder = '/content/gdrive/MyDrive/Big_Data/Project/Model/template'
app = Flask(__name__, template_folder=templates_folder)
run_with_ngrok(app)

@app.route("/", methods=['GET', 'POST'])
@app.route("/home", methods=['GET', 'POST'])
def home():
    if request.method == 'POST':
        url = request.form['url']
        try:
            spark = sparknlp.start()
            spark = SparkSession.builder.appName("sentimentanalysis").getOrCreate()
            url = url
            r = re.search(r"i\.(\d+)\.(\d+)", url)
            shop_id, item_id = r[1], r[2]
            ratings_url = "https://shopee.vn/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0"

            offset = 0
            d = {"username": [], "comment": []}
            flag_stop = False  # Biến cờ để kiểm tra điều kiện dừng vòng lặp
            while not flag_stop:
                data = requests.get(
                    ratings_url.format(shop_id=shop_id, item_id=item_id, offset=offset)
                ).json()

                if "data" not in data or "ratings" not in data["data"]:
                    print("Không có dữ liệu hoặc đã xảy ra lỗi.")
                    break

                ratings = data["data"]["ratings"]
                if not ratings:
                    print("Dữ liệu 'ratings' rỗng hoặc bằng None. Dừng lại và không lưu vào file CSV.")
                    break

                for rating in ratings:
                    comment_text = rating["comment"]

                    if comment_text is not None:  # Kiểm tra xem trường "comment" có giá trị hợp lệ hay không (không là None)
                        comment_text = comment_text.strip()

                    d["username"].append(rating["author_username"])
                    d["comment"].append(comment_text)

                    print(rating["author_username"])
                    print(comment_text)
                    print("-" * 100)

                offset += 20

            # Lưu vào file CSV chỉ khi có dữ liệu có trường "comment" không rỗng
            if d["comment"]:
                df = pd.DataFrame(d)
                df.to_csv("dataEN.csv", index=False)
            # Đọc dữ liệu từ file CSV vào DataFrame với tùy chọn multiLine=True để xử lý dữ liệu có nhiều dòng
            df = spark.read.csv("dataEN.csv", header=True, multiLine=True, inferSchema=True)

            # Đổi tên cột "comment" thành "short_review"
            df = df.withColumnRenamed("comment", "short_review")

            # Loại bỏ các dòng có trường "short_review" hoặc "username" rỗng từ DataFrame
            df = df.dropna(subset=["short_review", "username"])
            df = df.dropna(subset=["short_review"])
            # Xoá các icon (ký tự không phải tiếng Anh) trong trường "short_review"
            df = df.withColumn("short_review", regexp_replace("short_review", r"[^a-zA-Z\s]", " "))

            # Thay thế các chuỗi gồm nhiều khoảng trắng liên tiếp bằng một khoảng trắng duy nhất
            df = df.withColumn("short_review", regexp_replace("short_review", r"\s+", " "))

            # Loại bỏ khoảng trắng ở đầu và cuối mỗi giá trị trong trường "short_review"
            df = df.withColumn("short_review", trim(col("short_review")))

            # Lọc các dòng có trường "short_review" không rỗng
            df = df.filter(col("short_review") != "")

            # # Lưu lại DataFrame thành file CSV
            # df.write.csv("cleaned_dataEN.csv", header=True, mode="overwrite")
            # Load model and motional analysis  --------------------------------------------------------------------
            model_path = "/content/gdrive/MyDrive/Big_Data/Project/Model/best_model"
            loaded_model = PipelineModel.load(model_path)
            # Use the loaded model to make predictions on the new data
            predictions = loaded_model.transform(df)
            df = predictions.select(['short_review','label.result']).toPandas()
            df['result'] = df['result'].apply(lambda x: x[0])

            # Process name label
            # Function to map 'pos' to 'positive' and 'neg' to 'negative'
            def map_sentiment(sentiment):
                if sentiment == 'pos':
                    return 'positive'
                elif sentiment == 'neg':
                    return 'negative'
                else:
                    return sentiment

            # Applying the mapping function to the 'result' column
            df['result'] = df['result'].apply(map_sentiment)

            # Displaying the DataFrame
            # print(df)

            positive_count = df[df['result'] == 'positive'].shape[0]
            negative_count = df[df['result'] == 'negative'].shape[0]
            print(f"Number of positive reviews: {positive_count}")
            print(f"Number of negative reviews: {negative_count}")
            # Saving the Pandas DataFrame to a CSV file
            df.to_csv('predictions.csv', index=False)
            # Xử lý dữ liệu từ URL (đọc CSV và lấy dữ liệu)
            spark.stop()
            df = pd.read_csv('/content/predictions.csv')  # Đọc dữ liệu từ file CSV qua URL
            headers = df.columns.tolist()  # Lấy danh sách các cột làm tiêu đề
            rows = df.values.tolist()  # Lấy dữ liệu của từng dòng
            # Xử lý phân trang dữ liệu
            page = int(request.args.get('page', 1))
            items_per_page = 300
            start_idx = (page - 1) * items_per_page
            end_idx = min(start_idx + items_per_page, len(rows))
            rows = rows[start_idx:end_idx]
            # Xác định trang trước và trang kế tiếp
            prev_page = None
            next_page = None
            if page > 1:
                prev_page = f"?page={page - 1}"
            if end_idx < len(df):
                next_page = f"?page={page + 1}"

            return render_template('display.html', headers=headers, rows=rows, show_pagination=True,
                                   prev_page=prev_page, next_page=next_page,
                                   positive_count=positive_count, negative_count=negative_count)
        except Exception as e:
            return f"Lỗi: {e}"
    return render_template('index.html')

if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://a6dd-104-196-247-234.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [23/Jul/2023 15:26:34] "GET / HTTP/1.1" 200 -


a*****a
Value For Money:yes
Performance:press & hold to choose e option.
Best Feature(s):it works to count steps more accurately than a phone.

You hv to turn the watch towards you then it'll light up. Setting up was simple. And seller gave videos & photos on e how to go abt it. to charge is to put inside a charger head itself not thru a usb. Delivery abt a wk. Can change ur pic w ur own choice, but everytime u link to ur phone it changes back. 

But will buy another colour.
----------------------------------------------------------------------------------------------------
norman86
Performance:good
Value For Money:yes
Best Feature(s):very useful in sports and  monitoring health vitals

Good quality product, fast delivery and in good condition, will buy again.. thanks
----------------------------------------------------------------------------------------------------
c*****9
Best Feature(s):very good
Value For Money:excellent
Performance:very good

This product is cheap, good and easy 

INFO:werkzeug:127.0.0.1 - - [23/Jul/2023 15:30:24] "POST / HTTP/1.1" 200 -
